In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [3]:
import logging
import logging.config
import configparser
import os
import time
from pathlib import Path
import signal
from cachepath import CachePath, TempPath, Path
import requests
import shutil



import lmsquery
from waveshare_epd import epd5in83

In [4]:
import cfg
import epdlib

In [5]:
# this works best as a global variable
logConfig = Path(cfg.LOGCONFIG)
logging.config.fileConfig(logConfig.absolute())
# logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(name)s %(levelname)s: %(message)s')

logger = logging.getLogger(__name__)

In [6]:
appShortName = 'slimpi'
name = 'com.txoof.'
appLongName = name+appShortName

In [7]:
def configuration(configFile=None):
    
    configDefaults = cfg.CONFIGDEFAULTS
    
    if not configFile:
         configFile = Path(cfg.CONFIGFILE)

    config = configparser.ConfigParser()
    logger.info(f'reading configuration: {configFile}')
    config.read(configFile)
    
    try:
        for section in configDefaults:
            if section not in config.sections():
                logger.debug(f'adding section: {section}')
                config.add_section(section)
            
            for option in configDefaults[section]:
                if not config.has_option(section, option):
                    logger.debug(f'missing option: {option}')
                    logger.debug(f'setting {option} to: {configDefaults[section][option]}')
                    config[section][option] = str(configDefaults[section][option])
            with open(configFile, 'w') as file:
                config.write(file)
    
    except Exception as e:
        logging.exception(f'exception on configuration file: {e}')
        raise
    
    logger.debug(f'config file contains sections: {config.sections()}')
    return config

In [8]:
def query(lms, last=0, delay=7):
    '''query the player only when a specified delay has passed
    Accepts:
        last: float - last time query was called

    Returns:
        tuple(last, lms.now_playing()
    '''
    if last==0:
        last = time.clock_gettime(time.CLOCK_MONOTONIC)-delay

    if time.clock_gettime(time.CLOCK_MONOTONIC) > last+delay:
        return time.clock_gettime(time.CLOCK_MONOTONIC), lms.now_playing()

    else:
        return last, None

In [9]:
def cacheArt(query={}):
    if not query:
        return None
    
    cachePath = CachePath(appLongName, dir=True)
    required = ['artwork_url', 'album_id'] 
    missing = []
    
    for key in required:
        if not key in query:
            missing.append(key)
    if missing:
        logger.warning(f'required value(s) in `query` missing: {missing}')
        return None
    
    albumArtFile = cachePath/(query['album_id']+'.jpg')
    
    # check if file already exists in cache
    if albumArtFile.exists():
        logging.debug(f'album artwork already downloaded at: {albumArtFile}')
        return albumArtFile
    
    r = False
    albumArtURL = query['artwork_url']
    
    # try to fetch the album art
    try:
        r = requests.get(albumArtURL, stream=True)
    except requests.exceptions.RequestException as e:
        logging.error(f'failed to fetch artwork at {albumArtURL}: {e}')
        
    if r:
        try:
            with open(albumArtFile, 'wb') as outFile:
                shutil.copyfileobj(r.raw, outFile)
                logging.debug(f'wrote album art to: {albumArtFile}')
        except (OSError, FileExistsError, ValueError) as e:
            logging.error(f'failed to write {albumArtFile}: {e}')
    else:
        logging.error('skipping download of album artwork due to previous errors')

    return albumArtFile
    

In [10]:
class signalHandler(object):
    '''handle specific signals and allow graceful exiting while loop
    https://stackoverflow.com/a/31464349/5530152
    
    Signals Handled Gracefully:
        SIGINT
        SIGTERM
    Atributes:
        kill_now (bool) default: False
    '''
    kill_now = False
    def __init__(self):
        signal.signal(signal.SIGINT, self.exit_gracefully)
        signal.signal(signal.SIGTERM, self.exit_gracefully)
    
    def exit_gracefully(self, signum, frame):
        self.kill_now = True

In [11]:
def main():
    '''main entry point
    '''   
    logger.setLevel(logging.DEBUG)
    logger.info('Starting program')
    config = configuration()
    # scan for lms server and use the first one (this may be a terrible idea for other people)
#     lmsServer = lmsquery.scanLMS()[0]
    
    # define LMS query object
#     lms = lmsquery.LMSQuery(lmsServer['host'], lmsServer['port'], config.get('server', 'player'))
    lms = lmsquery.LMSQuery(player_id=config.get('server', 'player'))
    
    screen = epdlib.Screen()
    screen.epd = epd5in83.EPD()
    screen.initEPD()
#     screen.clearEPD()
    layout = epdlib.Layout(layout=epdlib.layouts.threeRow)
    layout.update_contents(query(lms, 0)[1])
    screen.initEPD()
    screen.elements=layout.blocks.values()
#     screen.concat()
#     screen.writeEPD()    
    
    
    sigHandler = signalHandler()
    
    # last update
    updated = 0
    # id of currently playing track
    nowPlayingID = None
    # status of player 
    nowPlayingMode = None

    # now playing fields to display
    fields = ['title', 'album', 'artist', 'mode', 'artwork_url']
    
    
    print(f'pid: {os.getpid()}')
    try:
        while not sigHandler.kill_now:
            # add some try: around this
            updated, value = query(lms=lms, last=updated, delay=3)
            
            response = value
            
            # if there's a response, check to see if display needs an update
            if response: 
                # if the album ID changed the player state changed, update the display
                if response['id'] != nowPlayingID or response['mode'] != nowPlayingMode:
                    
                    albumArt = cacheArt(response)
                    
                    if not albumArt:
                        albumArt = cfg.NOIMAGE
                        
                    value['coverart'] = str(albumArt)
                    
                    layout.update_contents(value)
                    screen.initEPD()
                    screen.elements=layout.blocks.values()
                    screen.concat()
                    screen.writeEPD()                      
                    
                    nowPlayingID = response['id']
                    nowPlayingMode = response['mode']
                    for field in fields:
                        print(f'{field}: {response[field]}')
                    print('\n')
            time.sleep(0.5)
    finally:
        print("cleaning up")
        screen.initEPD()
        screen.clearEPD()
        return layout    

In [12]:
if __name__ == '__main__':
    
    foo = main()

<ipython-input-11-7c80c73b8be7>:main:5:INFO - Starting program
<ipython-input-7-4275f2380fdc>:configuration:9:INFO - reading configuration: servercfg.ini
<ipython-input-7-4275f2380fdc>:configuration:30:DEBUG - config file contains sections: ['server', 'screen']
epd5in83:ReadBusy:69:DEBUG - e-Paper busy
epd5in83:ReadBusy:72:DEBUG - e-Paper busy release
Layout:layout:191:DEBUG - calculating values from layout for resolution (600, 448)
Layout:_calculate_layout:221:DEBUG - ***title***
Layout:_check_keys:108:DEBUG - checking key/values
Layout:_check_keys:113:DEBUG - missing key: dimensions; adding and setting to None
Layout:_calculate_layout:228:DEBUG - dimensions: (600, 179)
Layout:_calculate_layout:257:DEBUG - has explict position
Layout:_calculate_layout:259:DEBUG - abs_coordinates: (0, 0)
Layout:_scalefont:134:DEBUG - calculating font size
Layout:_scalefont:135:DEBUG - using font at path: /home/pi/src/slimpi_epd/fonts/Open_Sans/OpenSans-Bold.ttf
Layout:_scalefont:147:DEBUG - target X fo

Block:_text2image:344:DEBUG - text image dimensions: (117, 72)
Block:_text2image:369:DEBUG - v-center image coordinates
Block:_text2image:373:DEBUG - image coordinates (360, 368)
epd5in83:ReadBusy:69:DEBUG - e-Paper busy
epd5in83:ReadBusy:72:DEBUG - e-Paper busy release
pid: 715
<ipython-input-9-8dcb301e2ede>:cacheArt:20:DEBUG - album artwork already downloaded at: /tmp/com.txoof.slimpi/5126.jpg
Layout:update_contents:311:DEBUG - updating blocks
Layout:update_contents:320:DEBUG - ignoring block id
Layout:update_contents:317:DEBUG - updating block: title
Block:text_formatter:315:DEBUG - formatted list:
 ['Not Fighting', 'Anymore']
Block:_text2image:344:DEBUG - text image dimensions: (358, 152)
Block:_text2image:356:DEBUG - h-center line: Not Fighting
Block:_text2image:356:DEBUG - h-center line: Anymore
Block:_text2image:364:DEBUG - h-center image coordinates
Block:_text2image:369:DEBUG - v-center image coordinates
Block:_text2image:373:DEBUG - image coordinates (121, 14)
Layout:update_c

In [ ]:
    sc = epdlib.Screen()
    sc.epd = epd5in83.EPD()
    sc.initEPD()
    sc.clearEPD()

epd5in83:ReadBusy:69:DEBUG - e-Paper busy
epd5in83:ReadBusy:72:DEBUG - e-Paper busy release
epd5in83:ReadBusy:69:DEBUG - e-Paper busy


In [ ]:
tl = epdlib.Layout(layout=epdlib.layouts.clock)

In [ ]:
from PIL import Image

In [ ]:
i = Image.new('1', (600, 7), color=0)

In [ ]:
upd = {'time': '20:40', 'mode': 'stopped', 'bar': i}
tl.update_contents(upd)

In [ ]:
tl.blocks['time'].image

In [ ]:
sc.elements = tl.blocks.values()

In [ ]:
sc.concat()

In [ ]:
sc.initEPD()
sc.writeEPD()

In [ ]:
import datetime

In [ ]:
l = epdlib.Layout(resolution=(600, 392))

In [ ]:
l._scalefont(font='./fonts/Ubuntu/Ubuntu-Regular.ttf', dimensions=(600,392))

In [ ]:
b = epdlib.Block.TextBlock(font='./fonts/Ubuntu/Ubuntu-Regular.ttf', font_size=87, area=(600, 392))

In [ ]:
help(b)

In [ ]:
t = datetime.datetime.now().time()
timeStr = f'{t.hour}:{t.minute}'